In [ ]:
!pip install langchain-community==0.2.15 langchain-chroma==0.1.3 langchain-text-splitters==0.2.2 langchain-huggingface==0.0.3 langchain-groq==0.1.9 unstructured==0.15.0 unstructured[pdf]==0.15.0 nltk==3.8.1

In [ ]:
!apt-get install poppler-utils
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (746 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

In [ ]:
import os

from langchain_community.document_loaders import UnstructuredPDFLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [ ]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

··········


In [ ]:
loader = DirectoryLoader("data/", glob="./*.pdf", loader_cls=UnstructuredPDFLoader)
documents = loader.load()

In [ ]:
print(len(documents))

3


In [ ]:
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=500
)

text_chunks = text_splitter.split_documents(documents)

In [ ]:
persist_directory = "doc_db"

In [ ]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vectorstore = Chroma.from_documents(
    documents=text_chunks,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
query = "What does the document say about DELINEATING THE CLINICAL SYNDROME?"
response = qa_chain.invoke({"query":query})

In [ ]:
print(response["result"])

The document states that it is useful to divide Respiratory Tract Infections (RTIs) into those involving the upper and the lower tracts. 

The clinical syndromes involving the upper tract include:

- Otitis media
- Mastoiditis
- Sinusitis
- Pharyngitis

Infections of the lower tract can be divided into:

- Tracheobronchitis
- Bronchiolitis
- Pneumonia

Most of these conditions can exist in acute and chronic forms. Acute disease is usually caused by viral or bacterial infections, and chronic disease is usually caused by fungi, slow-growing bacteria such as mycobacteria, bacteria adapted to persist in biofilms, and occasional less common pathogens such as parasites. Chronic infections can also develop when structural changes occur as a result of recurrent or severe acute infections, surgical intervention, or other processes that alter the structural integrity of the respiratory tract.


In [ ]:
query = "What does the document say about Receptor-binding domain?"
response = qa_chain.invoke({"query":query})
print(response["result"])

The document discusses the Receptor-binding domain (RBD) of the SARS-CoV-2 spike glycoprotein in several sections. Here are the key points:

1. **Location and Structure**: The RBD is located on the S1 subunit of the spike glycoprotein and has a string of domains, including the N-terminal domain (NTD), subdomain 1 (SD1), and the RBD itself. The RBD harbors the ACE2-binding site, which lies across the top of the RBD, spanning the neck and shoulders.

2. **Conformation**: The RBD adopts a range of configurations on the spike, from 'up' to 'down', and only the up conformation can interact with ACE2.

3. **Binding of Neutralizing Antibodies**: The RBD is a major target for neutralizing antibodies, which can be grouped into several clusters based on their epitopes: left shoulder, neck, right shoulder, left flank, and right flank.

4. **Importance of RBD in Neutralization**: Most potent neutralizing antibodies induced by vaccination or natural infection target the RBD and usually interfere wi

In [ ]:
print(response["source_documents"])

[Document(metadata={'source': 'data/paper2.pdf'}, page_content='Despite the loss of activity of most neutralizing mAbs, a few have been reported recently that retain potency against Alpha, Beta, Gamma, Delta, BA.1, BA.2, and BA.4/5 variants, and in some cases BA.1.1 or BA.2.12.1. Based on their footprints on the RBD, these mAbs can be roughly divided into two groups: those targeting the back of the left-shoulder (F61 [27] and Omi-42 [28]) and right-shoulder binders (LY-CoV1404 (bebtelovimab) [29], 2–7 [30], XGv265 [31], XG005 [32], AZD1061 [1], MB.02 [33], SP1–77 [34], and 002-S21F2) [35]. However, it is likely that many of these mAbs will lose potency against the most recently emerging Omicron sublineages (BA.2.3.20, BA.2.75, BA.4.6, BA.5, BJ.1, BQ.1, and XBB) with one or several of five mutations: R346X, K444X, V445X, N450D, and N460X. Specifically, MAbs men- tioned above binding in the front of the right shoulder of the RBD may be knocked out by R346T/S, K444T/R, V445P, G446S, and N

In [ ]:
query = "What does the document say about  Asthma definition?"
response = qa_chain.invoke({"query":query})
print(response["result"])

According to the document, subjects were considered to have current asthma if they reported that they currently were using asthma medication and/or had had an attack of asthma in the past 12 months. Additionally, information on asthma symptoms such as wheeze and nocturnal symptoms within the last 12 months was also reported.


In [ ]:
print(response["source_documents"])

[Document(metadata={'source': 'data/paper3.pdf'}, page_content='(current\n\nsmoker)/No;\n\nIf No, are you an ex-smoker? Yes\n\nAsthma definition Subjects were considered to have current asthma if they reported that they currently were using asthma medication and/or had had an attack of asthma in the past 12 months [34, 35]. Information on asthma symptoms such as wheeze and nocturnal symptoms within the last 12 months was also reported.\n\nOccupational and environmental exposure The occupations reported by subjects were coded in accordance with the International Standard Classification of Occupations (ISCO-88) by an experienced occupational hygienist (ADH) [36]. These codes were connected to an asthma Job Exposure Matrix ( JEM) based on exposure patterns from the Nordic counties [32]. JEM classifications included high molecular weight agents (HMW), low molecular weight agents (LMW) and irritating agents [31]. In addition, we included self-reported exposure to dampness and mould defined 

In [ ]:
query = "how many shoulder shown in documents?"
response = qa_chain.invoke({"query":query})
print(response["result"])

In the provided documents, the term "shoulder" is mentioned in the context of the RBD (Receptor Binding Domain) of the SARS-CoV-2 virus. Specifically, it refers to the location where some neutralizing antibodies bind.

There are two mentions of "shoulder" in the documents:

1. "back of the left-shoulder" (referring to the location where F61 and Omi-42 antibodies bind)
2. "right-shoulder" (referring to the location where LY-CoV1404, 2–7, XGv265, XG005, AZD1061, MB.02, SP1–77, and 002-S21F2 antibodies bind)

So, there are two "shoulders" mentioned in the documents.


In [ ]:
query = "how many flank shown in documents?"
response = qa_chain.invoke({"query":query})
print(response["result"])

I don't know.


In [ ]:
query = "which flank shown in documents?"
response = qa_chain.invoke({"query":query})
print(response["result"])

According to Figure 1c in the document, the RBD has the following flanks:

- Left flank
- Right flank


In [ ]:
query = "what is mentioned about WUHAN in documents?"
response = qa_chain.invoke({"query":query})
print(response["result"])

In the provided documents, the term "Wuhan" is mentioned in the context of the Wuhan strain of the virus. Specifically, it is mentioned in the following sentence:

"Stem-helix region The stem-helix region (residues 1144–1158 of Wuhan strain) is part of the viral fusion machinery and is highly conserved among betacoronaviruses [49]."

This suggests that the Wuhan strain is a specific strain of the virus that was used as a reference point for studying the stem-helix region.


In [ ]:
query = "How do the colored regions in panels (d) and (e) indicate areas of interest in the protein structure? What role might these highlighted areas play in protein interactions?"
response = qa_chain.invoke({"query":query})
print(response["result"])

In panels (d) and (e) of Figure 3, the colored regions indicate areas of interest in the protein structure, specifically the SD1 region of the SARS-CoV-2 spike protein. The colors used are:

- Blue: P008_60
- Red: S3H3
- Green: SD1.040

These colors represent the binding patterns of the corresponding antibodies (P008_60, S3H3, and SD1.040) with the SD1 region of the spike protein. The highlighted areas in panels (d) and (e) suggest that these antibodies bind to specific regions of the SD1, which may play a crucial role in protein interactions.

The SD1 region is a part of the spike protein that is involved in the formation of the trimeric spike structure. The highlighted areas in panels (d) and (e) may indicate the binding sites of the antibodies, which could be involved in:

1. Stabilizing or destabilizing the spike trimer: The binding of antibodies to specific regions of the SD1 could either stabilize or destabilize the trimeric structure of the spike protein, affecting its ability t

In [ ]:
query = "give the interpretation of graphs present in this documents?"
response = qa_chain.invoke({"query":query})
print(response["result"])

Based on the provided data, I will attempt to interpret the graphs. However, please note that the actual graphs are not visible, and I will rely on the numerical data to make inferences.

**Graph 1: Socioeconomic index**

This graph appears to show the distribution of socioeconomic index (SEI) scores among different populations. The data is presented as a bar chart with six categories (1, 2, 3, 4, 5, and 6) and their corresponding frequencies.

* The most common SEI score is 2 (37.8%), followed by 1 (19.3%) and 3 (18.4%).
* The distribution of SEI scores varies across different populations, with some populations having a higher proportion of individuals with lower SEI scores (e.g., population 1 has a higher proportion of individuals with SEI score 1).

**Graph 2: Body mass index (BMI)**

This graph appears to show the distribution of BMI categories among different populations. The data is presented as a bar chart with four categories (<20, 20-25, 25-30, and >30) and their corresponding

In [ ]:
query = "explain graph of  Asthma and No asthma in documents?"
response = qa_chain.invoke({"query":query})
print(response["result"])

The graph provided in the document appears to be a series of odds ratios (OR) with 95% confidence intervals (CI) for the association between various factors and asthma. However, without a specific graph title or description, I will attempt to explain the general structure and content of the graph based on the provided data.

The graph seems to be a table or a series of tables, with each row representing a different factor or characteristic, and each column representing a different aspect of asthma. The factors or characteristics are listed on the left-hand side, and the aspects of asthma are listed at the top.

The factors or characteristics include:

1. Smoking status (Never-smoker, Ex-smoker, Current smoker)
2. Allergic rhinitis
3. Asthma
4. Hypertension
5. Heart disease
6. Diabetes
7. Children at home
8. Centre (location)

The aspects of asthma include:

1. Asthma (presence or absence)
2. Wheeze
3. Nocturnal symptoms
4. Asthma medication use
5. Asthma attacks in the past 12 months

